In [12]:
import pandas as pd
from rag import get_embedding
import numpy as np
import torch
import os
import torch.nn.functional as F
from langchain_openai.chat_models import ChatOpenAI 
import random  

In [8]:
df = pd.read_csv('smus_page.csv')
df.head()
df.shape

/opt/anaconda3/envs/voice-chat-python3-11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


torch.Size([1, 279, 768])

In [13]:
pages_and_text = []
for index, page in enumerate(df["Page Content"].tolist()):
    pages_and_text.append({"page number": index,
                           "page_char_count": len(page),
                           "page_word_count": len(page.split(" ")),
                           "page_sentence_count": len(page.split(".")),
                           "page_token_count": len(page)/4, # 1 token ~ 4 characters
                           "text": page})
random.sample(pages_and_text, 1)

[{'page number': 80,
  'page_char_count': 6501,
  'page_word_count': 1008,
  'page_sentence_count': 56,
  'page_token_count': 1625.25,
  'text': '   Breadcrumb Privacy Policy We at St. Michaels University School want you, our parents, students and staff, to be aware of how and why we handle your personal information. We work hard to respect and maintain your privacy. However, the very nature of our business is such that the collection, use and disclosure of personal information is fundamental to the services we provide. SMUS has adopted these privacy principles, which apply to the collection, use and disclosure of personal information. Personal information, for the purposes of these privacy principles, means information that identifies an individual. For example: an individual’s name, birth date, address, age, health and financial information is personal information which SMUS may collect, use and in certain circumstances, where necessary, disclose, in the course of carrying on busines

In [14]:
df = pd.DataFrame(pages_and_text)
df.head()

,page number,page_char_count,page_word_count,page_sentence_count,page_token_count,text
0,0,1496,241,14,374.00,Explore Cookie Settings When you visit any...
1,1,3488,567,30,872.00,Breadcrumb Start Here Thank you for choosin...
2,2,1169,194,10,292.25,Breadcrumb Admissions Publications If you ...
3,3,2809,462,20,702.25,Breadcrumb Middle School The Middle School ...
4,4,2167,345,15,541.75,Schaffter Hall for music (left) is home to ...


In [15]:
df.describe().round()

,page number,page_char_count,page_word_count,page_sentence_count,page_token_count
count,99.0,99.0,99.0,99.0,99.0
mean,49.0,3041.0,528.0,24.0,760.0
std,29.0,2867.0,497.0,27.0,717.0
min,0.0,162.0,23.0,1.0,40.0
25%,24.0,1180.0,196.0,10.0,295.0
50%,49.0,2517.0,467.0,17.0,629.0
75%,74.0,3641.0,665.0,29.0,910.0
max,98.0,18664.0,3216.0,180.0,4666.0
